In [1]:
%cd ..

C:\Users\hp\dev\video-membership-app


In [2]:
from app import db
from app.playlists.models import Playlist
from app.users.models import User
from app.videos.models import Video

from cassandra.cqlengine.management import sync_table

In [3]:
from app.indexing.client import get_index
from app.indexing.schemas import * # noqa

In [4]:
db.get_session()
sync_table(User)
sync_table(Video)
sync_table(Playlist)

In [5]:
import uuid
import json
from pydantic import BaseModel, Field, validator, root_validator
from typing import Optional

In [6]:
video_obj = dict(Video.objects.first())
video_obj

{'host_id': 'X3qUfOB3x8Q',
 'db_id': UUID('f299acc5-e6bd-11ed-bddb-c46516f58b27'),
 'host_service': 'youtube',
 'title': 'VS Code Day 2023',
 'url': 'https://www.youtube.com/watch?v=X3qUfOB3x8Q',
 'user_id': UUID('fd5b5178-e0e1-11ed-8672-c46516f58b27')}

In [7]:
class VideoIndexSchema(BaseModel):
    objectID: str = Field(alias="host_id")
    objectType: str = "Video"
    title: Optional[str]
    path: str = Field(alias="host_id")
        
    @validator("path")
    def set_path(cls, v, values, **kwargs):
        return f"/videos/{v}"

In [8]:
VideoIndexSchema(**video_obj).dict()

{'objectID': 'X3qUfOB3x8Q',
 'objectType': 'Video',
 'title': 'VS Code Day 2023',
 'path': '/videos/X3qUfOB3x8Q'}

In [9]:
playlist_obj = dict(Playlist.objects.first())
playlist_obj

{'db_id': UUID('31780f9f-e744-11ed-91fb-c46516f58b27'),
 'user_id': None,
 'updated': datetime.datetime(2023, 5, 6, 13, 29, 6, 185000),
 'host_ids': ['9C85o8jIgUU', '786VV6f_abM', '-_pgcFQ0l64', '786VV6f_abM'],
 'title': 'Hello Video'}

In [10]:
class PlaylistIndexSchema(BaseModel):
    objectID: uuid.UUID = Field(alias="db_id")
    objectType: str = "Playlist"
    title: Optional[str]
    path: uuid.UUID = Field(default="/")
        
#     @validator("path")
#     def set_path(cls, v, values, **kwargs):
#         return f"/playlists/{v}"
    
    @root_validator
    def set_defaults(cls, values):
        objectID = values.get("objectID")
        values["objectID"] = str(objectID)
        values["path"] = f"/playlists/{objectID}"
        return values

In [11]:
PlaylistIndexSchema(**playlist_obj).dict()

{'objectID': '31780f9f-e744-11ed-91fb-c46516f58b27',
 'objectType': 'Playlist',
 'title': 'Hello Video',
 'path': '/playlists/31780f9f-e744-11ed-91fb-c46516f58b27'}

In [12]:
json.loads(PlaylistIndexSchema(**playlist_obj).json())

{'objectID': '31780f9f-e744-11ed-91fb-c46516f58b27',
 'objectType': 'Playlist',
 'title': 'Hello Video',
 'path': '/playlists/31780f9f-e744-11ed-91fb-c46516f58b27'}

In [13]:
video_q = [dict(x) for x in Video.objects.all()]
videos_dataset = [VideoIndexSchema(**obj).dict() for obj in video_q]
videos_dataset

[{'objectID': 'X3qUfOB3x8Q',
  'objectType': 'Video',
  'title': 'VS Code Day 2023',
  'path': '/videos/X3qUfOB3x8Q'},
 {'objectID': 'RDE6Uz73A7g',
  'objectType': 'Video',
  'title': 'The Hunger Games: The Ballad of Songbirds & Snakes (2023) Official Trailer',
  'path': '/videos/RDE6Uz73A7g'},
 {'objectID': '9ix7TUGVYIo',
  'objectType': 'Video',
  'title': 'The Matrix Resurrections – Official Trailer 1',
  'path': '/videos/9ix7TUGVYIo'},
 {'objectID': 'itnqEauWQZM',
  'objectType': 'Video',
  'title': 'Transformers: Rise of the Beasts | Official Trailer (2023 Movie)',
  'path': '/videos/itnqEauWQZM'},
 {'objectID': '9C85o8jIgUU',
  'objectType': 'Video',
  'title': '#3 Try REACTJS Tutorial - Display Json Data',
  'path': '/videos/9C85o8jIgUU'},
 {'objectID': 'nNpvWBuTfrc',
  'objectType': 'Video',
  'title': 'The Matrix Resurrections – Official Trailer 2',
  'path': '/videos/nNpvWBuTfrc'},
 {'objectID': 'r51cYVZWKdY',
  'objectType': 'Video',
  'title': 'The Flash - Official Trailer 

In [14]:
playlist_q = [dict(obj) for obj in Video.objects.all()]
playlist_dataset = [PlaylistIndexSchema(**obj).dict() for obj in playlist_q]
playlist_dataset

[{'objectID': 'f299acc5-e6bd-11ed-bddb-c46516f58b27',
  'objectType': 'Playlist',
  'title': 'VS Code Day 2023',
  'path': '/playlists/f299acc5-e6bd-11ed-bddb-c46516f58b27'},
 {'objectID': '9e3f974f-e6ab-11ed-b591-c46516f58b27',
  'objectType': 'Playlist',
  'title': 'The Hunger Games: The Ballad of Songbirds & Snakes (2023) Official Trailer',
  'path': '/playlists/9e3f974f-e6ab-11ed-b591-c46516f58b27'},
 {'objectID': '20eba022-e669-11ed-9ab3-010101010000',
  'objectType': 'Playlist',
  'title': 'The Matrix Resurrections – Official Trailer 1',
  'path': '/playlists/20eba022-e669-11ed-9ab3-010101010000'},
 {'objectID': '9538e354-e6ab-11ed-bb2f-c46516f58b27',
  'objectType': 'Playlist',
  'title': 'Transformers: Rise of the Beasts | Official Trailer (2023 Movie)',
  'path': '/playlists/9538e354-e6ab-11ed-bb2f-c46516f58b27'},
 {'objectID': '47386fb1-eaa4-11ed-af23-c46516f58b27',
  'objectType': 'Playlist',
  'title': '#3 Try REACTJS Tutorial - Display Json Data',
  'path': '/playlists/473

In [20]:
dataset = playlist_dataset + videos_dataset
dataset

[{'objectID': 'f299acc5-e6bd-11ed-bddb-c46516f58b27',
  'objectType': 'Playlist',
  'title': 'VS Code Day 2023',
  'path': '/playlists/f299acc5-e6bd-11ed-bddb-c46516f58b27'},
 {'objectID': '9e3f974f-e6ab-11ed-b591-c46516f58b27',
  'objectType': 'Playlist',
  'title': 'The Hunger Games: The Ballad of Songbirds & Snakes (2023) Official Trailer',
  'path': '/playlists/9e3f974f-e6ab-11ed-b591-c46516f58b27'},
 {'objectID': '20eba022-e669-11ed-9ab3-010101010000',
  'objectType': 'Playlist',
  'title': 'The Matrix Resurrections – Official Trailer 1',
  'path': '/playlists/20eba022-e669-11ed-9ab3-010101010000'},
 {'objectID': '9538e354-e6ab-11ed-bb2f-c46516f58b27',
  'objectType': 'Playlist',
  'title': 'Transformers: Rise of the Beasts | Official Trailer (2023 Movie)',
  'path': '/playlists/9538e354-e6ab-11ed-bb2f-c46516f58b27'},
 {'objectID': '47386fb1-eaa4-11ed-af23-c46516f58b27',
  'objectType': 'Playlist',
  'title': '#3 Try REACTJS Tutorial - Display Json Data',
  'path': '/playlists/473

In [15]:
index = get_index()

In [16]:
dir(index)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_chunk',
 '_config',
 '_create_temporary_name',
 '_name',
 '_raw_batch',
 '_sync',
 '_transporter',
 'app_id',
 'batch',
 'browse_objects',
 'browse_rules',
 'browse_synonyms',
 'clear_objects',
 'clear_rules',
 'clear_synonyms',
 'copy_to',
 'delete',
 'delete_by',
 'delete_object',
 'delete_objects',
 'delete_rule',
 'delete_synonym',
 'exists',
 'find_object',
 'get_object',
 'get_object_position',
 'get_objects',
 'get_rule',
 'get_settings',
 'get_synonym',
 'get_task',
 'move_to',
 'name',
 'partial_update_object',
 'partial_update_objects',
 'replace_all_objects',
 'replace_all_rules',
 'replace_a

In [22]:
idx_response = index.save_objects(dataset).wait()
len(idx_response[0]["objectIDs"])

18

In [23]:
index.search("matrix")

{'hits': [{'objectType': 'Video',
   'title': 'The Matrix Resurrections – Official Trailer 2',
   'path': '/videos/nNpvWBuTfrc',
   'objectID': 'nNpvWBuTfrc',
   '_highlightResult': {'objectType': {'value': 'Video',
     'matchLevel': 'none',
     'matchedWords': []},
    'title': {'value': 'The <em>Matrix</em> Resurrections – Official Trailer 2',
     'matchLevel': 'full',
     'fullyHighlighted': False,
     'matchedWords': ['matrix']},
    'path': {'value': '/videos/nNpvWBuTfrc',
     'matchLevel': 'none',
     'matchedWords': []}}},
  {'objectType': 'Playlist',
   'title': 'The Matrix Resurrections – Official Trailer 2',
   'path': '/playlists/def30719-e668-11ed-9049-010101010000',
   'objectID': 'def30719-e668-11ed-9049-010101010000',
   '_highlightResult': {'objectType': {'value': 'Playlist',
     'matchLevel': 'none',
     'matchedWords': []},
    'title': {'value': 'The <em>Matrix</em> Resurrections – Official Trailer 2',
     'matchLevel': 'full',
     'fullyHighlighted': Fals